# Feature Engineering of MusicBox 
since we have date information and analyze churn rate, we can set activity frequency and recency for each user's behavior(log type)

**No enough memory to loadin all three file in Pandas **, so maybe still have to use spark
## First for play.log
since we have done downsampling based on user_id in play.log, we first do frequency generation for sample_play.log
### Loadin sample_play

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#% matplotlib inline

In [7]:
print(pd.__version__)

0.22.0


In [2]:


#sparksession is used to create DF, register DF as tables, excute SQL over tables, cache tables
from pyspark.sql import SparkSession

from pyspark.sql import Row
from pyspark.sql import functions

from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql.types import *

from operator import add
from operator import ge
import sys

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
#Next, create pyspark.sql.SparkSession, which is the main entry point for DataFrame and SQL functionality
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [6]:
colname = ['device', 'song_id','song_type', 'play_time', 'song_length', 'play_date', 'uid', 'Churn']
datatype = {'device': 'object', 'song_id': 'float64', 'song_type': 'float64', 'song_length': 'float64', 'date':'object', 'uid': 'float64', 'Churn': 'float64'}
sampled_play = pd.read_csv("D:/MusicFile/sampled_play_new.csv", header = 0, dtype = datatype, names = colname)

MemoryError: 

In [4]:
sampled_play.head()

,device,song_id,song_type,play_time,song_length,play_date,uid,Churn
0,ar,3385963.0,1.0,235868.0,235.0,2017-03-01,154422624.0,1.0
1,ar,4623962.0,0.0,243.0,243.0,2017-03-01,154408096.0,1.0
2,ar,1967689.0,0.0,139.0,275.0,2017-03-01,154421168.0,1.0
3,ip,3224320.0,0.0,256.0,257.0,2017-03-01,154414400.0,1.0
4,ar,292409.0,0.0,0.0,220.0,2017-03-01,154422032.0,1.0


In [5]:
#convert type of date from object to datetime
sampled_play['play_date'] = pd.to_datetime(sampled_play['play_date'])

In [6]:
sampled_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6118322 entries, 0 to 6118321
Data columns (total 8 columns):
device         object
song_id        float64
song_type      float64
play_time      float64
song_length    float64
play_date      datetime64[ns]
uid            float64
Churn          float64
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 350.1+ MB


In [7]:
sampled_play.describe()

,song_id,song_type,play_time,song_length,uid,Churn
count,6.118322e+06,6.118322e+06,6.118322e+06,6.118322e+06,6.118322e+06,6.118322e+06
mean,1.746825e+14,9.836472e-02,1.360537e+04,2.674886e+02,1.640897e+08,2.157386e-01
std,4.616627e+16,3.082414e-01,2.942228e+07,2.919063e+02,1.474736e+07,4.113338e-01
min,-1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.009040e+05,0.000000e+00
25%,5.432730e+05,0.000000e+00,7.000000e+00,2.070000e+02,1.676631e+08,0.000000e+00
50%,4.184588e+06,0.000000e+00,1.510000e+02,2.440000e+02,1.681440e+08,0.000000e+00
75%,7.042074e+06,0.000000e+00,2.450000e+02,2.800000e+02,1.685898e+08,0.000000e+00
max,1.844674e+19,2.000000e+00,7.276968e+10,6.685100e+04,1.692483e+08,1.000000e+00


### loadin cleaned_search/download.log

In [2]:
colnames = ['uid', 'device', 'file_name', 'search_date', 'search_time']
stype = {'uid': 'float64', 'device': 'object', 'file_name': 'object', 'search_date': 'object', 'search_time':'object'}
cleaned_search = pd.read_csv("D:/MusicFile/cleaned_search_1.csv", header = 0, dtype= stype)
cleaned_search.columns = colnames
cleaned_search['search_date'] = pd.to_datetime(cleaned_search['search_date'])

In [3]:
cleaned_search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9794977 entries, 0 to 9794976
Data columns (total 5 columns):
uid            float64
device         object
file_name      object
search_date    datetime64[ns]
search_time    object
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 261.6+ MB


In [14]:
cleaned_search.head()

,uid,device,file_name,search_date,search_time
0,154000000.0,ip,1_1_search.log,3/1/2017,2017-03-01T00:00:24.000-06:00
1,154000000.0,ar,1_1_search.log,3/1/2017,2017-03-01T00:00:53.000-06:00
2,154000000.0,ip,1_1_search.log,3/1/2017,2017-03-01T00:00:54.000-06:00
3,154000000.0,ar,1_1_search.log,3/1/2017,2017-03-01T00:00:55.000-06:00
4,154000000.0,ar,1_1_search.log,3/1/2017,2017-03-01T00:00:55.000-06:00


In [4]:
colnamed = ['uid', 'device', 'song_id', 'download_date']
downtype = {'uid': 'float64','device': 'object', 'song_id': 'float64', 'date':'object'}
cleaned_download = pd.read_csv("D:/MusicFile/cleaned_download_1.csv", header = 0, dtype = downtype)
cleaned_download.columns = colnamed
cleaned_download['download_date'] = pd.to_datetime(cleaned_download['download_date'])

In [25]:
cleaned_download.columns

Index(['uid', 'device', 'song_id', 'download_date'], dtype='object')

In [5]:
cleaned_download.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7727762 entries, 0 to 7727761
Data columns (total 4 columns):
uid              float64
device           object
song_id          float64
download_date    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 206.4+ MB


In [15]:
cleaned_download.head()

,uid,device,song_id,download_date
0,168000000.0,ar,442554.0,2017-03-30
1,168000000.0,ar,6334611.0,2017-03-30
2,168000000.0,ar,9867382.0,2017-03-30
3,168000000.0,ar,6660691.0,2017-03-30
4,168000000.0,ar,157606.0,2017-03-30


In [19]:
search_uid = pd.Series(list(cleaned_search['uid'].unique()))
download_uid = pd.Series(list(cleaned_download['uid'].unique()))
play_uid = pd.Series(list(sampled_play['uid'].unique()))

In [23]:
print("How many users have search record:", len(search_uid))
print("How many users have download record:", len(download_uid))
#print("How many users in sampled_play have search record:", search_uid.isin(play_uid)


How many users have search record: 459
How many users have download record: 403


In [22]:
search_uid.isin(play_uid).sum()

3

### Combine sampled_play, cleaned_download, cleaned_search into one file, then create frequency/recency feature; but we also can filter sampled_uid from cleaned_download, cleaned_search, then create features within each type, finally combine, which might run faster

In [38]:
from pyspark import StorageLevel
MEMORY_AND_DISK_SER = StorageLevel(True, True, False, False, 1)

In [ ]:
#reload sampled_play in spark

#sampled_play = spark.createDataFrame(sampled_play)
'''
colname = ['device', 'song_id','song_type', 'play_time', 'song_length', 'play_date', 'uid', 'Churn']
datatype = {'device': 'object', 'song_id': 'float64', 'song_type': 'float64', 'song_length': 'float64', 'date':'object', 'uid': 'float64', 'Churn': 'float64'}
sampled_play = pd.read_csv("D:/MusicFile/sampled_play_new.csv", header = 0, dtype = datatype, names = colname)
'''
schemap = StructType([StructField('device', StringType(), False),
                     StructField('song_id', FloatType(), False),
                     StructField('song_type', IntegerType(), False),
                     StructField('play_time', FloatType(), False),
                     StructField('song_length', IntegerType(), False),
                     StructField('date', StringType(), False),
                     StructField('uid', FloatType(), False), 
                     StructField('Churn', IntegerType(), False)])
#sampled_play = spark.createDataFrame(play, schemap)

In [7]:
sampled_play = spark.read.csv("D:/MusicFile/sampled_play_new.csv", header = True, inferSchema=True)
print(sampled_play.printSchema())
#sampled_play = sampled_play.withColumnRenamed("date", "play_date")
pd.DataFrame(sampled_play.take(10), columns = sampled_play.columns)
#inferSchema is right, no need to set schema by ourselves

root
 |-- device: string (nullable = true)
 |-- song_id: double (nullable = true)
 |-- song_type: double (nullable = true)
 |-- play_time: double (nullable = true)
 |-- song_length: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- uid: double (nullable = true)
 |-- Churn: integer (nullable = true)

None


,device,song_id,song_type,play_time,song_length,date,uid,Churn
0,ar,3385963.0,1.0,235868.0,235.0,2017-03-01,154422624.0,1
1,ar,4623962.0,0.0,243.0,243.0,2017-03-01,154408096.0,1
2,ar,1967689.0,0.0,139.0,275.0,2017-03-01,154421168.0,1
3,ip,3224320.0,0.0,256.0,257.0,2017-03-01,154414400.0,1
4,ar,292409.0,0.0,0.0,220.0,2017-03-01,154422032.0,1
5,ar,9811955.0,0.0,5.0,245.0,2017-03-01,154422368.0,1
6,ar,450444.0,0.0,206.0,206.0,2017-03-01,154422496.0,1
7,ar,5237384.0,0.0,125.0,125.0,2017-03-01,154422624.0,1
8,ip,129962.0,0.0,64.0,317.0,2017-03-01,154422496.0,1
9,ar,6684281.0,0.0,215.0,278.0,2017-03-01,153043808.0,1


In [22]:
sampled_play.take(164)

[Row(device='ar', song_id=3385963.0, song_type=1.0, play_time=235868.0, song_length=235.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154422624.0, Churn=1),
 Row(device='ar', song_id=4623962.0, song_type=0.0, play_time=243.0, song_length=243.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154408096.0, Churn=1),
 Row(device='ar', song_id=1967689.0, song_type=0.0, play_time=139.0, song_length=275.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154421168.0, Churn=1),
 Row(device='ip', song_id=3224320.0, song_type=0.0, play_time=256.0, song_length=257.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154414400.0, Churn=1),
 Row(device='ar', song_id=292409.0, song_type=0.0, play_time=0.0, song_length=220.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154422032.0, Churn=1),
 Row(device='ar', song_id=9811955.0, song_type=0.0, play_time=5.0, song_length=245.0, date=datetime.datetime(2017, 3, 1, 0, 0), uid=154422368.0, Churn=1),
 Row(device='ar', song_id=450444.0, song_type=0.0, play_time

In [5]:
#reload in cleaned_search
cleaned_search = spark.read.csv("D:/MusicFile/cleaned_search_1.csv", header = True, inferSchema=True).cache()
print(cleaned_search.printSchema())
cleaned_search = cleaned_search.withColumnRenamed("record_date", "date")
pd.DataFrame(cleaned_search.take(10), columns = cleaned_search.columns)

root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- file_name: string (nullable = true)
 |-- record_date: timestamp (nullable = true)
 |-- record_time: timestamp (nullable = true)

None


,uid,device,file_name,date,record_time
0,154436640.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:24
1,154407264.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:53
2,154407856.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:54
3,154407248.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:55
4,154407328.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:55
5,154407248.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:56
6,154407264.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:00:59
7,154407264.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:00:59
8,154407552.0,ip,1_1_search.log,2017-03-01,2017-03-01 00:01:00
9,154407248.0,ar,1_1_search.log,2017-03-01,2017-03-01 00:01:02


In [6]:
#reload in cleaned_download

cleaned_download = spark.read.csv("D:/MusicFile/cleaned_download_1.csv", header = True, inferSchema=True).cache()
print(cleaned_download.printSchema())
#for the convience of following step, change date in each log into 'date'
#cleaned_download = cleaned_download.withColumnRenamed("date", "download_date")
pd.DataFrame(cleaned_download.take(10), columns = cleaned_download.columns)


root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- song_id: double (nullable = true)
 |-- date: timestamp (nullable = true)

None


,uid,device,song_id,date
0,168019808.0,ar,442554.0,2017-03-30
1,168019808.0,ar,6334611.0,2017-03-30
2,168019808.0,ar,9867382.0,2017-03-30
3,168019808.0,ar,6660691.0,2017-03-30
4,168019808.0,ar,157606.0,2017-03-30
5,168019808.0,ar,3372481.0,2017-03-30
6,168019808.0,ar,3216525.0,2017-03-30
7,168019808.0,ar,6427523.0,2017-03-30
8,168019808.0,ar,6538686.0,2017-03-30
9,168019808.0,ar,9327383.0,2017-03-30


In [ ]:
'''
search = sc.textFile("D:/MusicFile/cleaned_search_1.csv")
schems = StructType([StructField('uid', FloatType(), False),
                     StructField('device', StringType(), True),
                     StructField('search_time', StringType(), False),
                     StructField('url', StringType(), True),
                     StructField('file_name', StringType(), True)])
'''

### Create Frequency feature
set frequency based on 1-day, 3-day, 7-day, 14-day, 21-day for each type of log file, the snapshot_time is 2017-04-29

In [14]:
types = ['play', 'download', 'search']
windows = [1,3,7,14,21]

end_time = '2017-04-29'
dd = spark.createDataFrame([('2017-04-29',)], ['ddd'])
dd.select(to_date(dd.ddd).alias('snapshot_date')).collect()
dd.withColumn('newd', date_sub(dd.ddd, 1)).show()
#dd.schema

+----------+----------+
|       ddd|      newd|
+----------+----------+
|2017-04-29|2017-04-28|
+----------+----------+



In [56]:
to_date(lit('2017-04-29'))
#dd.withColumn('datediffer',datediff('20170429', '20170301')).show()

Column<b'to_date(2017-04-29)'>

In [51]:
def get_freq(df, event_type, window, snapshot_date):
    #date_sub(date startdate, tinyint/smallint/int days)
    df = df.withColumn('string_date', df.date.cast(StringType()).substr(1,10))
    #df3 = df2.withColumn('start_date', date_sub(df.snapshot_date, window))
    #df4 = df3.filter((df3.start_date <= df3.date) & (df3.date < snapshot_date))
    df2 = df.filter(datediff(snapshot_date, df.string_date) <= window)
    #df_feature = df4.groupby('uid').agg(func.count('uid').alias('freq_'+ event_type + '_last_' + str(window) + 'days'))
    #df_feature.head()
    return df2
    #return df_feature

In [52]:
#test for 1-day frequency in play.log
print("Test for :", types[0], "log_type of", windows[0],"- day frequency before", end_time)
play_freq = get_freq(sampled_play, types[0], windows[0], end_time)
play_freq.take(5)

Test for : play log_type of 1 - day frequency before 2017-04-29


AnalysisException: "cannot resolve '`2017-04-29`' given input columns: [play_time, Churn, song_length, song_id, uid, date, string_date, device, song_type];;\n'Filter (datediff('2017-04-29, string_date#401) <= 1)\n+- Project [device#164, song_id#165, song_type#166, play_time#167, song_length#168, date#169, uid#170, Churn#171, substring(cast(date#169 as string), 1, 10) AS string_date#401]\n   +- Relation[device#164,song_id#165,song_type#166,play_time#167,song_length#168,date#169,uid#170,Churn#171] csv\n"

In [ ]:
df_feature_list = []
for event in types:
    for window in window_List:
        df_feature_list.append(get_freq(df, event, window, snapshot_time))